In [14]:
import bs4
import random
from lxml import etree

import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from time import sleep

from webdriver_manager.chrome import ChromeDriverManager

In [30]:
df=pd.read_csv("data/all_manila_hotels_v2.csv")
links = df.url
links = links.head(3)
# links.tolist()
# links
links[0]

'https://www.booking.com/hotel/ph/midas-hotel-and-casino.html?label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAEJuAEXyAEM2AEB6AEB-AELiAIBqAIDuALa9-ebBsACAdICJDNiMWUxMTlhLTc3ODEtNGI3NC05MWVlLTQ5MDA0MDQzYzY5NNgCBuACAQ&sid=b3a20c12be6b3d4d9b1b0a71b11d8898&aid=304142&ucfs=1&arphpl=1&checkin=2022-12-01&checkout=2022-12-02&dest_id=-2437894&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&nflt=ht_id%3D204&srpvid=7ca3572c9035002e&srepoch=1669119834&all_sr_blocks=63717502_357003256_2_41_0&highlighted_blocks=63717502_357003256_2_41_0&matching_block_id=63717502_357003256_2_41_0&sr_pri_blocks=63717502_357003256_2_41_0__438520&tpi_r=2&from=searchresults#hotelTmpl'

In [53]:
def review_details(dom,main_dict,current_page):
    country_list=[i for i in dom.xpath("//div[@class='c-review-block']//div[1]//div[@class='c-review-block__row c-review-block__guest']//div[@class='bui-avatar-block__text']//span[@class='bui-avatar-block__subtitle']//text()") if i!='\n']
    for count,i in enumerate(dom.xpath("//div[@class='c-review-block']")):
        
        try:
            country=country_list[count]
        except:
            country=""
        inner_dict={}
        #Left Block
        
        try:
            name=i.findall("div")[1].findall("div//span[@class='bui-avatar-block__title']")[0].text
        except:
            name=""
        
        try:
            room_type=i.findall("div")[1].findall("div")[0].findall("div[@class='c-review-block__row c-review-block__room-info-row']//ul/li//a//div")[0].text
        except:
            room_type=""
        
        try:
            nights_stayed=i.findall("div")[1].findall("div")[0].findall("ul[@class='bui-list bui-list--text bui-list--icon bui_font_caption c-review-block__row c-review-block__stay-date']//li//div")[0].text
        except:
            nights_stayed=""
            
        try:
            month_stayed=i.findall("div")[1].findall("div")[0].findall("ul[@class='bui-list bui-list--text bui-list--icon bui_font_caption c-review-block__row c-review-block__stay-date']//li//div//span")[0].text
        except:
            month_stayed=""
            
            
        try:
            occupant_type=i.findall("div")[1].findall("div")[0].findall("ul[@class='bui-list bui-list--text bui-list--icon bui_font_caption review-panel-wide__traveller_type c-review-block__row']//li//div")[0].text
        except:
            occupant_type=""
        
        try:
            date_reviewed=i.findall("div")[1].findall("div//span[@class='c-review-block__date']")[0].text
        except:
            date_reviewed=""
        
        try:
            review_score=i.findall("div")[1].findall("div")[1].findall("div")[0].findall("div[@class='bui-grid']")[0] \
              .findall("div[@class='bui-grid__column-1 bui-u-text-right']//div//div")[0].text
        except:
            review_score=""
        
        #Right Block
        
        try:
            short_review=i.findall("div")[1].findall("div//h3")[0].text
        except:
            short_review=""
            
        try:
            positive_review=i.findall("div")[1].findall("div")[1].findall("div")[1].findall("div//div[@class='c-review__row']" +
                            "//p//span[@class='c-review__body']")[0].text
        except:
            positive_review=""

        try:
            negative_review=i.findall("div")[1].findall("div")[1].findall("div")[1].findall("div//div[@class='c-review__row lalala']" +
                         "//p//span[@class='c-review__body']")[0].text
        except:
            negative_review=""

        inner_dict={"name":name, "room_type":room_type, "nights_stayed":nights_stayed, "month_stayed":month_stayed,
                    "occupant_type":occupant_type, "from_country":country,"review_score":review_score,
                   "date_reviewed":date_reviewed,"short_review":short_review,"positive_review":positive_review,
                    "negative_review":negative_review
                   }

        main_dict["From Page "+str(current_page)+", Person- "+ str(count+1)]=inner_dict




def get_review_body(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    driver.maximize_window()


    time.sleep(3)
    driver.execute_script('window.scrollTo(0, 100)')
        
    #click Read all reviews button
    

    
    
    driver.find_element(by="xpath", value="//a[@data-target= 'hp-reviews-sliding']").click()
    #Switch to review window
    driver.switch_to.window(driver.window_handles[0])

    soup = BeautifulSoup(driver.page_source,"html.parser")

    #get min and max pages
    current_page = 1
    time.sleep(3)
    last = driver.find_element(by="xpath", value="//*[@id='review_list_page_container']/div[4]/div/div[1]/div/div[2]/div/div[7]/a/span[2]").text
    last2 = last.split(" ")
    last_page = int(last2[1])
    print("Total Pages- " + str(last_page))
    review_body=[]
    main_dict={}
    while current_page <= last_page:
        print("Current Page- " +str(current page))
        time.sleep(3)
        
        page_source=driver.page_source
        soup=bs4.BeautifulSoup(page_source,'lxml')
        dom = etree.HTML(str(soup))
        
        
        review_details(dom,main_dict,current_page)
        
        time.sleep(3)
        if current_page<last_page:
            driver.find_element(by="xpath", value="//a[contains(@class, 'pagenext')]").click()
        current_page +=1
    
    driver.quit()
    dict_to_df={"url":url,"All_reviews":main_dict}
    df=pd.DataFrame({k: pd.Series([v]) for k,v in dict_to_df.items()})
    return df


SyntaxError: invalid syntax (2376800835.py, line 109)

# Run on multiple URL's

In [59]:
from tqdm import tqdm
#df=pd.read_csv("data/all_manila_hotels.csv")
#df=pd.read_csv("data/all_manila_hotels_v2.csv")
df=pd.read_csv("data/consolidated_urls.csv")
links=df.link

df_hotels_consolidated=pd.DataFrame()

start_index=3
end_index=6

for count,i in tqdm(enumerate(links[start_index:end_index])):
    print("Opening link- "+ str(count) + " " + str(i))
    #print(df.hotel_name[count+start_index])
    #try:
    df_to_append=get_review_body(i)
    df_hotels_consolidated=pd.concat([df_hotels_consolidated,df_to_append])
        
    # except Exception as e: 
    #     print("")
    #     print("Error:")
    #     print(str(e))
    #     print("Skipping due to error")
    #     continue
        
    print(" ")

0it [00:00, ?it/s]

Opening link- 0 https://www.booking.com/hotel/ph/seda-residences-makati.html?label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAEJuAEXyAEM2AEB6AEB-AELiAIBqAIDuALa9-ebBsACAdICJDNiMWUxMTlhLTc3ODEtNGI3NC05MWVlLTQ5MDA0MDQzYzY5NNgCBuACAQ&sid=b3a20c12be6b3d4d9b1b0a71b11d8898&aid=304142&ucfs=1&arphpl=1&checkin=2022-12-01&checkout=2022-12-02&dest_id=-2437894&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=4&hapos=104&sr_order=popularity&nflt=ht_id%3D204&srpvid=7ca3572c9035002e&srepoch=1669119861&all_sr_blocks=465341501_203119067_0_1_0&highlighted_blocks=465341501_203119067_0_1_0&matching_block_id=465341501_203119067_0_1_0&sr_pri_blocks=465341501_203119067_0_1_0__556325&from=searchresults#hotelTmpl


1it [03:31, 211.86s/it]

 
Opening link- 1 https://www.booking.com/hotel/ph/go-hotels-manila-airport-road.html?label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAEJuAEXyAEM2AEB6AEB-AELiAIBqAIDuALa9-ebBsACAdICJDNiMWUxMTlhLTc3ODEtNGI3NC05MWVlLTQ5MDA0MDQzYzY5NNgCBuACAQ&sid=b3a20c12be6b3d4d9b1b0a71b11d8898&aid=304142&ucfs=1&arphpl=1&checkin=2022-12-01&checkout=2022-12-02&dest_id=-2437894&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=6&hapos=106&sr_order=popularity&nflt=ht_id%3D204&srpvid=7ca3572c9035002e&srepoch=1669119861&all_sr_blocks=175599003_261884663_0_0_0&highlighted_blocks=175599003_261884663_0_0_0&matching_block_id=175599003_261884663_0_0_0&sr_pri_blocks=175599003_261884663_0_0_0__184400&from=searchresults#hotelTmpl


2it [08:57, 278.70s/it]

 
Opening link- 2 https://www.booking.com/hotel/ph/vivere-hotel-and-resorts.html?label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAEJuAEXyAEM2AEB6AEB-AELiAIBqAIDuALa9-ebBsACAdICJDNiMWUxMTlhLTc3ODEtNGI3NC05MWVlLTQ5MDA0MDQzYzY5NNgCBuACAQ&sid=b3a20c12be6b3d4d9b1b0a71b11d8898&aid=304142&ucfs=1&arphpl=1&checkin=2022-12-01&checkout=2022-12-02&dest_id=-2437894&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=7&hapos=107&sr_order=popularity&nflt=ht_id%3D204&srpvid=7ca3572c9035002e&srepoch=1669119861&all_sr_blocks=63717401_232584350_0_0_0&highlighted_blocks=63717401_232584350_0_0_0&matching_block_id=63717401_232584350_0_0_0&sr_pri_blocks=63717401_232584350_0_0_0__298893&from=searchresults#hotelTmpl


2it [10:12, 306.06s/it]


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x0080ACD3+2075859]
	Ordinal0 [0x0079EE61+1633889]
	Ordinal0 [0x0069B680+571008]
	Ordinal0 [0x006CBBD6+768982]
	Ordinal0 [0x006C1C76+728182]
	Ordinal0 [0x006E731C+881436]
	Ordinal0 [0x006C15BF+726463]
	Ordinal0 [0x006E7534+881972]
	Ordinal0 [0x006FB56A+963946]
	Ordinal0 [0x006E7136+880950]
	Ordinal0 [0x006BFEFD+720637]
	Ordinal0 [0x006C0F3F+724799]
	GetHandleVerifier [0x00ABEED2+2769538]
	GetHandleVerifier [0x00AB0D95+2711877]
	GetHandleVerifier [0x0089A03A+521194]
	GetHandleVerifier [0x00898DA0+516432]
	Ordinal0 [0x007A682C+1665068]
	Ordinal0 [0x007AB128+1683752]
	Ordinal0 [0x007AB215+1683989]
	Ordinal0 [0x007B6484+1729668]
	BaseThreadInitThunk [0x764AFA29+25]
	RtlGetAppContainerNamedObjectPath [0x76EF7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x76EF7A6E+238]


In [55]:
df_hotels_consolidated

,url,All_reviews
0,https://www.booking.com/hotel/ph/solaire-resor...,"{'From Page 1, Person- 1': {'name': 'Khadir', ..."


In [38]:
import pickle


output = open('data/sample_output_reviews.pkl', 'wb')
pickle.dump(df, output)
output.close()

df.to_csv("data/sample_output_reviews.csv")